In [324]:
import torch
from dpm.models import pca, PCA
import numpy as np
from scipy.stats import ortho_group

In [325]:
def build_toy_dataset(N, D, K, sigma=1):
    x_train = np.random.normal(0.0, 1.0, size=(N, D))
    w = ortho_group.rvs(K)[:, :D]
    print(w)
    return torch.tensor(x_train).float(), torch.tensor(w).float()

N = 5000  # number of data points
D = 1  # data dimensionality
K = 2  # latent dimensionality

x_train, w = build_toy_dataset(N, D, K)
x_train.shape, w.shape

[[-0.41764553]
 [-0.90861004]]


(torch.Size([5000, 1]), torch.Size([2, 1]))

In [326]:
def pca(X, k=2):
    if not isinstance(X, torch.Tensor):
        X = torch.tensor(X).float()
    X = X - X.mean(dim=0, keepdim=True)
    U, S, V = torch.svd(X)
    return torch.mm(X, V[:k].t())

In [327]:
x_train = x_train - x_train.mean(0)

In [328]:
x_train.mean(0), w.shape

(tensor([1.2946e-08]), torch.Size([2, 1]))

In [329]:
x = x_train.mm(w.t())
x.shape

torch.Size([5000, 2])

In [330]:
ret = pca(x, 1)


In [331]:
ret, _ = torch.sort(ret)

In [332]:
x_train, _ = torch.sort(x_train)

In [333]:
(ret.abs() - x_train.float().abs()).pow(2).sum()

tensor(7.1716e-12)

In [334]:
m = PCA(1)

In [335]:
ret = m.fit_transform(x)

In [336]:
x_train, _ = torch.sort(x_train)

In [337]:
ret, _ = torch.sort(ret)

In [338]:
(ret.abs() - x_train.float().abs()).pow(2).sum()

tensor(5.9584e-12)